# Getting started with langchain - question answering

This notebook walks through how to use LangChain for question answering over a given document. 

Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. However, using these LLMs in isolation is often insufficient for creating a truly powerful app - the real power comes when you can combine them with other sources of computation or knowledge.

And that is what we will do here!

### Sources
- GitHub: https://github.com/hwchase17/langchain
- Documents: https://python.langchain.com/en/latest/use_cases/question_answering.html
- PyPi: https://pypi.org/project/langchain/

### Contents
0. Install packages
1. Prepare data
2. Search the document
2. Call the LLM to do generative AI

## 0. Install packages

In [1]:
!pip install langchain
!pip install openai

## 1. Settings

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
import config

In [3]:
#store your openai api key as an environment variable - as required by OpenAI (see previous tutorial)
import os
import config # I've created a config.py file that stores my password
os.environ['OPENAI_API_KEY'] = config.openai_key #but don't keep it in your source code :-)

## 2. Get the data and create embeddings

In [4]:
#we will use the wget library to download the file 
import wget
filename = wget.download('https://github.com/hwchase17/chat-your-data/blob/master/state_of_the_union.txt')
filename

-1 / unknown

'state_of_the_union (1).txt'

In [5]:
#split the document into chunks
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

#create the embeddings
embeddings = OpenAIEmbeddings()

## 2. Search the document
A first step can be to search your document. This way we will find the location in the document where words of the query are mentioned. The result of this search is the relevant part of the document
(In the next step we will use this result and feed it to the LLM so it can generate a nice response.)

In [6]:
#first we will create docsearch variable. 
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

Using embedded DuckDB without persistence: data will be transient


In [7]:
#first tr
query = "What did the president say about Justice Breyer"
docs = docsearch.get_relevant_documents(query)

In [8]:
#print the best result
docs[0]

Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '31'})

## 3. Call the LLM to generate an answer
If you just want to get started as quickly as possible, this is the fasted way to do it.

In [9]:
#import two more liberies
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [10]:
#create the chain, give the pro
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
#create your prompt
query = "What did the president say about Justice Breyer"
chain.run(input_documents=docs, question=query)

' The president said that Justice Breyer is an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court, and thanked him for his service.'

## Conclusion
That's it! We've taken a text document from the web and did document search with it. Also we used a Large Language Model to generate a good answer for it!